In [2]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [3]:
# !pip install PyYAML

In [19]:
# load yaml
with open('data.yaml', mode='r')as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)


['tree']


In [20]:
# Load yolo model
yolo = cv2.dnn.readNetFromONNX('runs\\train\\Final_Model\\weights\\best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [21]:
# Load image
img = cv2.imread('Test_Model_Images/test4_2.jpg')
image = img.copy()
row, col, d = image.shape

# get the yolo prediction from the image
# 1) Convert image to square matrix(array)
max_rc = max(int(row), int(col))
input_image = np.zeros((max_rc, max_rc,3), dtype = np.uint8)
input_image[0:int(row), 0:int(col)] = image

# 2) get prediction from square array
input_wh_yolo = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255,(input_wh_yolo,input_wh_yolo),swapRB=True, crop = False)
yolo.setInput(blob)
preds = yolo.forward()  #detection or prediction from yolo

In [22]:
print(preds.shape)

(1, 25200, 6)


In [23]:
# Non Maximum Supression
# 1) Filter detectioon based on confidence (0.2) and probability score (0.1)
detection = preds[0]
boxes = []
confidences = []
classes = []


# width and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/input_wh_yolo
y_factor = image_h/input_wh_yolo

for i in range(len(detection)):
    row = detection[i]
    confidence = row[4] #confidence of detection an object
    if confidence > 0.2:
        class_score = row[5:].max() #Max probility of objects
        class_id = row[5:].argmax() # Get the index position at which  max prob occur
 
        if class_score > 0.3:
            cx, cy, w, h = row[0:4]
            # construct bounding box from four values
            # Left, top, width, height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)

            box = np.array([left, top, width, height])

            # Append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
            
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.1, 0.1).flatten()


In [24]:
# Number of objects detected by model.
print(len(index))

74


In [25]:
# Draw the boundings
for ind in index:
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]

    # To create rectangle
    # cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),1)
    
    # To create circle
    cv2.circle(image, (int(x+0.5*w), int(y+0.5*h)), radius = 10, color = (0,255,0), thickness=2)



In [27]:
# boxes
cv2.imshow('Original', img)
image = cv2.putText(image, 'Count of Trees: ' + str(len(index)),(5,20), cv2.FONT_HERSHEY_SIMPLEX,  
                   0.6, (0,0,255), 1, cv2.LINE_AA) 
cv2.imshow('Prediction', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
